In [1]:
from MECO_data_split import MECODataSplit, concat_MECO_langs

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:
df = concat_MECO_langs('ru', path_to_data='../Datasets/DataToUse/')
dataset = MECODataSplit(df)
X, y = dataset.split_by_unique_values(split_cols=['SubjectID'],
                                      include_cols='fix',
                                      test_size=0, fix_threshold=0,
                                      resample='truncate', series_length=800)

In [3]:
from keras.utils import to_categorical

y = to_categorical(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [17]:
# X, y = dataset.split_by_unique_values(split_cols=['Sentence'], include_cols='fix', test_size=0, fix_threshold=0,
#                                       resample='truncate', series_length=10)

import pandas as pd

pd.unique(dataset.data['Sentence'])

array(['Samuel Morse, best k', 'While he was always',
       'In 1825, Morse was i', 'Morse rushed to his',
       'The message had take', 'Grief-stricken, he g',
       'Leonardo da Vinci is', 'Even though most of',
       'The parachute, the m', 'Over five hundred ye',
       'The Amazon Rainfores', 'Areas of the rainfor',
       'However, it is slowl', 'First, a significant',
       'There are several co', 'Secondly, the rainfo',
       'Without rainforests,', 'Howard Gardner was a',
       'Basically, the theor', 'This is because peop',
       'He argued that there', 'One type of intellig',
       'People who possess t', 'Intrapersonal intell',
       'They enjoy thinking', 'People who have ling',
       'These people usually', 'The other types of i',
       'The Internet has mad', 'There are several pr',
       'Hackers can steal pe', 'One of the most obvi',
       'Passwords are also a', 'First, they should b',
       'Your first or last n', 'Longer passwords are',
       'Finall

In [12]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [13]:
from keras import Sequential
from keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense

def train_model(conv_filters=64, kernel_size=3, strides=1, dropout_rate=0.5, pool_size=2, epochs=10, batch_size=32):
    model = Sequential()
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, strides=strides,
                     activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, strides=strides, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

    return model

In [14]:
cnn = train_model(epochs=15, batch_size=32, dropout_rate=0.5)

Epoch 1/15


2023-03-02 11:45:26.185583: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 1s 24ms/step - loss: 603.9550 - accuracy: 0.3478
Epoch 2/15
3/3 [==============================] - 0s 25ms/step - loss: 512.6327 - accuracy: 0.5652
Epoch 3/15
3/3 [==============================] - 0s 36ms/step - loss: 1097.1665 - accuracy: 0.6667
Epoch 4/15
3/3 [==============================] - 0s 48ms/step - loss: 573.4393 - accuracy: 0.6667
Epoch 5/15
3/3 [==============================] - 0s 38ms/step - loss: 112.2166 - accuracy: 0.6522
Epoch 6/15
3/3 [==============================] - 0s 31ms/step - loss: 49.3346 - accuracy: 0.3768
Epoch 7/15
3/3 [==============================] - 0s 30ms/step - loss: 0.9865 - accuracy: 0.8841
Epoch 8/15
3/3 [==============================] - 0s 29ms/step - loss: 1.7757 - accuracy: 0.8841
Epoch 9/15
3/3 [==============================] - 0s 33ms/step - loss: 2.3665 - accuracy: 0.8696
Epoch 10/15
3/3 [==============================] - 0s 34ms/step - loss: 0.5235 - accuracy: 0.9420
Epoch 11/15
3/3 [===========

In [15]:
loss, accuracy = cnn.evaluate(X_test, y_test)

1/1 [==============================] - 0s 110ms/step - loss: 6.8694 - accuracy: 0.6667
